In [1]:
pip install aiohttp


In [54]:
pip install lxml

In [29]:
import time
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import requests
import ssl
from aiohttp import ClientSession
from itertools import  chain
import json

In [3]:
url = 'https://markets.businessinsider.com/index/components/s&p_500?p='


# Синхронный код

In [4]:
%%time 
names = []
for i in range(1, 11):
  request = requests.get(url+ f"{i}")
  print(request.status_code)
  soup = BeautifulSoup(request.text, 'html.parser').find(class_ ='table-small').find_all('a')
  for s in soup:
    names.append(s.get('title'))


200
200
200
200
200
200
200
200
200
200
CPU times: user 1.22 s, sys: 33.2 ms, total: 1.25 s
Wall time: 1.56 s


# Multithreads

In [5]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor

In [6]:
def fetch_response(url: str) -> requests.Response:
    return requests.get(url)

def names(request):
  names = []
  soup = BeautifulSoup(request.text, 'html.parser').find(class_ ='table-small').find_all('a')
  for s in soup:
    names.append(s.get('title'))
  return names

In [7]:
urls = []
for i in range(1, 11):
  urls.append(url+ f"{i}")




In [8]:
%%time 
with ThreadPoolExecutor(max_workers=10) as pool:
    responses = pool.map(fetch_response, urls)
    names = list(pool.map(names, responses))

CPU times: user 1.38 s, sys: 46.4 ms, total: 1.43 s
Wall time: 1.36 s


# AsyncIO + AIOHTTP

In [9]:
#Без этого кода не запускаются циклы async loop
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

Спарсим все 10 страниц сайта и сохраним в одну переменную

In [10]:

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.text()


async def run():
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for i in range(1, 11):
            url = f'https://markets.businessinsider.com/index/components/s&p_500?p={i}'
            task = asyncio.ensure_future(fetch(url.format(i), session))
            tasks.append(
                task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses

start = time.time()
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run())
main_pages = loop.run_until_complete(future) # Получаем 10 главных страаниц сайта
end = time.time()
print('******', end-start)

****** 0.07325458526611328


Названия компаний

In [11]:
def company_names(main_pages):
  comp_names = [] 
  soup = BeautifulSoup(main_pages, 'html.parser').find(class_ ='table-small').find_all('a')
  for s in soup:
    comp_names.append(s.get('title')) #Получаем имена компаний с главных страниц
  return comp_names 

Ссылки на страницы каждой компании

In [12]:
def sub_pages_links(main_pages):
  sub_pages = []
  soup = BeautifulSoup(main_pages, 'html.parser').find(class_ ='table-small').find_all('a')
  for s in soup:
    sub_pages.append(s.get('href'))
  return sub_pages

Получим имена и ссылки, распраралелим функции на потоки

In [13]:
%%time 
with ThreadPoolExecutor(max_workers=10) as pool:
    comp_names = list(chain.from_iterable(pool.map(company_names, main_pages)))
    sub_pages = list(chain.from_iterable(pool.map(sub_pages_links, main_pages)))

CPU times: user 2.42 s, sys: 43.1 ms, total: 2.47 s
Wall time: 2.47 s


Так же как для 10 главных страниц, спарсим страницы каждой компании и сохраним в переменную

In [14]:

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.text()


async def run():
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for i in sub_pages:
            url = f'https://markets.businessinsider.com{i}'
            
            task = asyncio.ensure_future(fetch(url.format(i), session))
            tasks.append(
                task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


start = time.time()
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run())
sub_pages_texts = loop.run_until_complete(future) # Страницы всех компаний
end = time.time()
print('******', end-start)

****** 29.14627432823181


In [64]:
def get_usd_course():
  request = requests.get("http://www.cbr.ru/scripts/XML_daily.asp")
  soup = BeautifulSoup(request.text, "lxml").find(id ="R01235").value
  return float((soup.text).replace(',' ,'.'))

get_usd_course()

76.2491

Текущая стоимость в рублях (конвертацию производить по текущему курсу, взятому с сайта центробанка РФ)

In [71]:
def get_prices(sub_pages_texts):
  prices = []
  soup = BeautifulSoup(sub_pages_texts, 'html.parser').find(class_ ='price-section__current-value')
  for s in soup:
    prices.append(float(s.strip().replace(',', ''))*get_usd_course())
  return prices

Код компании (справа от названия компании на странице компании)

In [66]:
def company_code(sub_pages_texts):
  codes = []
  
  soup = BeautifulSoup(sub_pages_texts, 'html.parser').find('span', class_ ='price-section__category')
  codes.append(soup.text.replace('Stock , ','').strip())
  return codes


P\E ratio

In [67]:
def get_p_e_ratio(sub_pages_texts):
  p_e_ratio = []
  soup = BeautifulSoup(sub_pages_texts, 'html.parser').find('div', class_ ='snapshot').find_all(class_ = 'snapshot__data-item')
  for s in soup:
    if s.find(class_= 'snapshot__header', text ='P/E Ratio' ):
      p_e_ratio.append(s.text[:11].strip())
      return p_e_ratio
  p_e_ratio.append(None)
  return p_e_ratio





Годовой рост/падение компании в процентах (основная таблица)

In [68]:
def get_year_growth(main_pages):
  growth_perc = []
  table = BeautifulSoup(main_pages, 'html.parser').find(class_ =('table-small'))
  for rows in table.find_all('tr')[1:]:
    growth_perc.append(rows.find_all(class_ = 'text-right')[6]
                       .find_all(class_ = ('colorRed', 'colorGreen'))[1]
                       .text)
  return growth_perc

      

Высчитать какую прибыль принесли бы акции компании (в процентах), если бы они были куплены на уровне 52 Week Low и проданы на уровне 52 Week High (справа от графика на странице компании)

In [72]:

def potential_profit(sub_pages_texts):
  profit = [] 
  try:
    week_high = (BeautifulSoup(sub_pages_texts, 'html.parser').find(class_ = 'snapshot__highlow-container').find_all(class_ = 'snapshot__highlow')[1]
                                                  .find_all('div', class_ =('snapshot__data-item snapshot__data-item--small snapshot__data-item--right',
                                                                        'snapshot__data-item snapshot__data-item--small')))   
    low = float(week_high[0].text[:15].strip().replace(',', ''))
    high = float(week_high[1].text[:15].strip().replace(',', ''))
    profit.append((high - low)*get_usd_course())
  except IndexError:
    profit.append(None)
  return profit




In [73]:
%%time 
with ThreadPoolExecutor(max_workers=100) as pool:
    potential_profit = list(chain.from_iterable(pool.map(potential_profit, sub_pages_texts)))
    year_growth = list(chain.from_iterable(pool.map(get_year_growth, main_pages)))
    p_e_ratio = list(chain.from_iterable(pool.map(get_p_e_ratio, sub_pages_texts)))
    company_code = list(chain.from_iterable(pool.map(company_code, sub_pages_texts)))
    price = list(chain.from_iterable(pool.map(get_prices, sub_pages_texts)))


CPU times: user 5min 18s, sys: 30 s, total: 5min 48s
Wall time: 5min 21s


Сохранить итоговую информацию в 4 JSON файла:

1. Топ 10 компаний с самими дорогими акциями в рублях.
2. Топ 10 компаний с самым низким показателем P/E.
3. Топ 10 компаний, которые показали самый высокий рост за последний год
4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.


Преобразование данных
 * Создадим таблицу со всеми данными
 * Затем будем ее сортировать по необходимой колонке
 * Запишем результаты в JSON

In [74]:
import pandas as pd
import numpy as np


In [75]:
print(len(comp_names), len(company_code), len(p_e_ratio), len(year_growth), len(potential_profit))

429 429 429 429 429


In [76]:
df = pd.DataFrame({'Name': comp_names,
                   'Code': company_code,
                   'Price, RUB': price,
                   'P/E': p_e_ratio,
                    'Year growth, %': year_growth,
                    'Potential profit, RUB': potential_profit
                   })

In [77]:
df.head(30)

,Name,Code,"Price, RUB",P/E,"Year growth, %","Potential profit, RUB"
0,3M,MMM,15182.720792,19.91,35.46%,5226.875805
1,AO Smith,AOS,5175.026417,25.06,66.31%,2454.458529
2,Abbott Laboratories,ABT,9505.975297,29.76,37.87%,3230.674367
3,AbbVie,ABBV,8281.414751,9.91,31.73%,2615.344130
4,Accenture,ACN,21848.417114,32.81,60.09%,9332.127349
5,Activision Blizzard,ATVI,7338.213384,26.40,47.15%,3216.949529
6,Adobe,ADBE,39412.397299,47.10,53.55%,16139.646997
7,Advance Auto Parts,AAP,14681.001714,18.27,62.23%,6377.474724
8,AES,AES,2160.899494,15.98,101.78%,1377.821237
9,Aflac,AFL,4083.901796,8.88,38.42%,1783.466449


1. Топ 10 компаний с самими дорогими акциями в рублях.

In [79]:
df_most_expensive = df.sort_values(by=['Price, RUB'], ascending=False)
result = df_most_expensive[:10].to_json(orient="records")
parsed = json.loads(result)
with open('most_expensive.json', 'w') as f:
    json.dump(parsed, f)


2. Топ 10 компаний с самым низким показателем P/E.

In [80]:
df_min_p_e_value = df.sort_values(by=['P/E'])
df_min_p_e_value[:10]
result = df_min_p_e_value[:10].to_json(orient="records")
parsed = json.loads(result)
with open('min_p_e_value.json', 'w') as f:
    json.dump(parsed, f)



3. Топ 10 компаний, которые показали самый высокий рост за последний год

In [82]:
df_max_year_growth = df.sort_values(by=['Year growth, %'], ascending=False)
df_max_year_growth[:10]
result = df_max_year_growth[:10].to_json(orient="records")
parsed = json.loads(result)
with open('max_year_growth.json', 'w') as f:
    json.dump(parsed, f)

4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.

In [83]:
df_most_valid = df.sort_values(by=['Potential profit, RUB'], ascending=False)
df_most_valid[:10]
result = df_most_valid[:10].to_json(orient="records")
parsed = json.loads(result)
with open('most_valid.json', 'w') as f:
    json.dump(parsed, f)